# Tunier

Damit unterschiedliche Algorithmen und deren Einstellungen vergleichen werden können, wird eine virtuelles Tunier implementiert.
Bei diesem Tunier nehmen die Algorithmen in verschiedenen Ausführungen teil und spielen mehrmals gegeneinander, damit sicher gestellt wird, dass es sich nicht um Zufall handelt.

Innerhalb eines Tuniers (eng. `Tournament`) spielen alle Teilnehmer gegen jeden anderen Teilnehmer, sodass jeder Teilnehmer einmal als Spieler weiß beginnt. Diese Teilnehmerbegegnungen nennen sich Runden (eng. `Round`) in denen mehrere Spiele (eng. `Match`) gespielt werden.

Am Ende des Tuniers kann dann anhand der Anzahl der Gewinne ausgewertet werden, welcher Algorithmus mit welcher Einstellung am besten abschneidet.

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Zunächst werden beide implementierten Algorithmen geladen: `AlphaBetaPruning` und `Minimax`.

In [ ]:
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-minimax.ipynb

Um eine übersichtlichere Entwicklung zu ermöglichen, werden Typdefinitionen geladen, welche später im Code verwendet werden.

In [ ]:
from typing import Optional, Union, List, Callable

## Spiel

Ein Spiel wird durch die Klasse `Match` implementiert, welche ein einziges Spiel zwischen zwei Teilnehmern darstellt. Der Konstruktor der Klasse erwartet zwei verpflichtende Argumente und sechs optionale Argumente:

Verpflichtend:
* `white` ist eine Instanz einer `ArtificialIntelligence`, die den weißen Spieler spielen wird;
* `black` ist eine Instanz einer `ArtificialIntelligence`, die den schwarzen Spieler spielen wird;

Optional:
* `start_state` ist der Startzustand, der verwendet werden soll, standardmäßig `s0`;
* `start_player` ist der Spieler, der das Spiel beginnen soll, standardmäßig `w`;
* `max_turns` ist die maximale Anzahl an Zügen, die das Spiel dauern darf, bevor es in einem Remis endet, standardmäßig `250`;
* `max_state_replayed` ist die maximale Anzahl, die eine Stellung nochmal gespielt werden darf, bevor das Spiel in einem Remis endet, standardmäßig `5`;
* `max_moves_without_mill` ist die maximale Anzahl von aufeinanderfolgenden Zügen in denen kein Stein geschlagen wurde, bevor das Spiel in einem Remis endet, standardmäßig `30`;
* `name` ein optionaler Name für das Spiel.

Des Weiteren werden zwei Attribute initialisiert, die für den Spielverlauf nötig sind:
* `log` der Verlauf aller Zustände;
* `no_mill_played` die Anzahl der letzten Züge ohne eine neue Mühle.

In [ ]:
class Match():
    def __init__(
        self,
        white: ArtificialIntelligence, black: ArtificialIntelligence,
        start_state = s0, start_player = 'w',
        max_turns: int = 250, max_state_replayed: int = 5, max_moves_without_mill: int = 30,
        name: str = ""
    ):
        self.white = white
        self.black = black
        
        self.state = start_state
        self.player = start_player
        self.max_turns = max_turns
        self.max_state_replayed = max_state_replayed
        self.max_moves_without_mill = max_moves_without_mill
        self.name = name
        
        self.log = [start_state]
        self.no_mill_played = 0

Für Entwicklungszwecke wird eine Stringdarstellung für die Klasse `Match` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
def __repr__(self: Match):
    return f"Match(name='{self.name}', white={type(self.white).__name__}, " + \
           f"black={type(self.black).__name__}, max_turns={self.max_turns}, " + \
           f"max_state_replayed={self.max_state_replayed}, max_moves_without_mill={max_moves_without_mill})"

Match.__repr__ = __repr__
del __repr__

Das Ergebnis eines Spiels wird in der `MatchResult` Klasse gespeichert. Durch die Aufteilung in die Klassen `Match` und `MatchResult` kann der Garbage Collector die `Match` Instanz löschen, sobald das Spiel beendet ist und die Variable nicht mehr verwendet wird. So können möglicherweise große Transpositionstabellen gelöscht werden und der RAM wieder frei gegeben werden.

Ein `MatchResult` besteht aus drei Attributen, die im Konstruktor gesetzt werden müssen:
* `winner` $\in Player \cup \{'\ '\}$, der Gewinner (oder Remis) des Spieles;
* `log` ist die chronologische Liste aller gespielen Zustände;
* `reason` eine Zeichenkette die genauer beschreibt warum das Spiel endete.

Für Entwicklungszwecke wird hier ebenfalls eine Stringdarstellung für die Klasse `MatchResult` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
class MatchResult():
    def __init__(self, winner: str, log: List, reason: str):
        self.winner = winner
        self.log    = log
        self.reason = reason
    
    def __repr__(self):
        return f"MatchResult(winner='{self.winner}', log={len(self.log): >3}, reason='{self.reason}')"

Die Hilfsfunktion `current_ai` gibt für das aktuelle Spiel die KI-Instanz zurück, die gerade am Zug ist.

In [ ]:
def current_ai(self: Match) -> ArtificialIntelligence:
    if self.player == 'w':
        return self.white
    return self.black

Match.current_ai = current_ai
del current_ai

Die Hilfsfunktion `check_draw` überprüft, ob der aktuelle Zustand und Spieler zu einem Remis führen. Ist dies der Fall, wird ein entsprechendes `MatchResult` mit Begründung, ansonsten `None`, zurückgegeben. Gründe für ein Remis sind:
* die maximale Anzahl an Zügen wurde gespielt - `max_turns`;
* eine Stellung wurde öfter als maximal erlaubt gespielt - `max_state_replayed`;
* die maximale Anzahl an Zügen ohne eine neue Mühle wurde gespielt - `max_moves_without_mill`.

In [ ]:
def check_draw(self: Match) -> Optional[MatchResult]:
    if len(self.log) >= self.max_turns:
        return MatchResult(
            winner = ' ',
            log    = self.log,
            reason = f"Reached max_turns after {self.max_turns} turns"
        )
    
    if self.log.count(self.state) >= self.max_state_replayed:
        return MatchResult(
            winner = ' ',
            log    = self.log,
            reason = f"State has been replayed for {self.log.count(self.state)} times"
        )
    
    if self.no_mill_played >= self.max_moves_without_mill:
        return MatchResult(
            winner = ' ',
            log    = self.log,
            reason = f"No mill has been played for {self.no_mill_played} moves"
        )
    return None

Match.check_draw = check_draw
del check_draw

Um ein Spiel zu spielen wird die Funktion `play` implementiert, diese bedient sich der im `Match` gespeicherten Einstellungen und Daten.

Prinzipiell unendlich lang sind die Spieler abwechselnd am Zug und spielen ihren am besten berechneten Zug. Ein Spiel endet sobald per `finished` Funktion ein Gewinner ermittelt wurde oder per `check_draw` Hilfsfunktion ein Remis beschlossen wurde. Ist dies nicht der Fall, wird der aktuelle Teilnehmer nach dem nächsten (besten) Zug befragt, dieser wird gespeichert und der Gegner ist am Zug.

Da nach einer undefinierten Anzahl an Zügen entweder `finished` oder `check_draw` ein Ergebnis liefern, wird immer eine `MatchResult` Instanz zurückgegeben.

In [ ]:
def play(self: Match) -> MatchResult:
    while True:
        draw = self.check_draw()
        if draw is not None:
            return draw
        
        if finished(self.state, self.player):
            # Draw was already checked
            winner = self.player if utility(self.state, self.player) == 1 else opponent(self.player)
            return MatchResult(
                winner = winner,
                log    = self.log,
                reason = f"A player won the match"
            )
        
        mills_before = findMills(self.state[1], self.player)
        
        bestMoves = self.current_ai().bestMoves(self.state, self.player)
        self.state  = bestMoves.choice()
        self.player = opponent(self.player)
        
        self.log.append(self.state)
        if playerPhase(self.state, self.player) != 1 and \
           countNewMills(self.state[1], mills_before, self.player) <= 0:
            self.no_mill_played += 1
        else:
            self.no_mill_played = 0

Match.play = play
del play

## Runde
Damit zufällige Gewinne möglichst ausgeschlossen werden, können mehrere Spiele zwischen den gleichen Teilnehmern gleichzeitig in einer Runde gespielt werden. Hierzu wird die Python Bibliothek `multiprocessing` verwendet, die für jedes Spiel einen eigenen Prozess startet.

In [ ]:
from multiprocessing import Pool

Die Klasse `Round` spiegelt solch eine Runde wieder und besitzt drei Parameter im Konstruktor:
* `white` ist eine Instanz oder eine Funktion die eine `ArtificialIntelligence` produziert, die den weißen Spieler spielt;
* `black` ist eine Instanz oder eine Funktion die eine `ArtificialIntelligence` produziert, die den schwarzen Spieler spielt;
* `instances` ist die Anzahl der Spiele die gleichzeitig gestartet werden sollen;
* `seed_offset` ist die Zahl, die auf den Seed addiert werden soll.

Die Parameter `white` und `black` können auch Funktionen akzeptieren, damit sichergestellt werden kann, dass die Teilnehmer innerhalb der gestarteten Spiele sich keine Transpositionstabllen teilen und somit einen Vorteil erhalten könnten.

In [ ]:
class Round():
    def __init__(
        self,
        white: Union[ArtificialIntelligence, Callable],
        black: Union[ArtificialIntelligence, Callable],
        instances: int,
        seed_offset: int
    ):
        self.white = white
        self.black = black
        self.instances = instances
        self.seed_offset = seed_offset

Für Entwicklungszwecke wird eine Stringdarstellung für die Klasse `Round` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
def __repr__(self: Round):
    return f"Round(white={type(self.white).__name__}, black={type(self.black).__name__}, instances={self.instances})"

Round.__repr__ = __repr__
del __repr__

Die Hilfsfunktion `execute` wird durch die Python Bibliothek `multiprocessing` innerhalb des neu erzeugten Prozesses ausgeführt. Sie setzt den Seed der Random-Funktion auf den Index der Runde damit verschiedene Spiele stattfinden und startet das Spiel.

In [ ]:
def execute(match):
    seed, rnd = match
    random.seed(seed)
    return rnd.play()

Die Funktion `play` erzeugt eine Liste von `instances` Spielen und erstellt gegebenenfalls die Instanzen der KIs aus den gespeicherten Erstellungsfunktionen. Daraufhin werden die Spiele per `multiprocessing` und `execute` Hilfsfunktion in neuen Prozessen gestartet.

Sobald jedes Spiel beendet wurde, werden die Ergebnisse wieder als Liste zurück gegeben.

In [ ]:
def play(self: Round) -> List[MatchResult]:
    matches = [
        Match(
            self.white() if callable(self.white) else self.white,
            self.black() if callable(self.black) else self.black,
            name = f"r={i: <2}"
        )
        for i in range(self.instances)
    ]
    with Pool(self.instances) as pool:
        return pool.map(execute, ((seed+self.seed_offset, match) for seed, match in enumerate(matches)))

Round.play = play
del play

## Tunier
Damit nun verschiedene Teilnehmer in unterschiedlichen Konstellationen verglichen werden können, wurde ein Tunier implementiert. Die Klasse `Tournament` erwartet einen verflichtenden und zwei optionale Parameter:

Verpflichtend:
* `participants` ist eine Liste an Teilnehmer `ArtificialIntelligence` Instanzen oder Funktionen, die diese erzeugen;

Optional:
* `instances_per_round` ist die Anzahl der Spiele die pro Runde und pro Teilnehmer Konstellation gespielt werden soll;
* `name` ist ein Name, der den Log-Dateien angehängt wird;
* `skip` ist die Anzahl der Runden, die übersprungen werden sollen. Dies ist Hilfreich falls die Berechnungen abbrechen und fortgesetzt werden sollen;
* `seed_offset` ist die Zahl, die auf den Seed einer Runde addiert werden soll. So können Runden in Teilen ausgerechnet werden.

Pro Spiel können durch die Transpositionstabelle bis zu 6 Gigabyte an RAM benötigt werden. Dementsprechend wird empfohlen die `instances_per_round` Einstellung zu bearbeiten.

In [ ]:
class Tournament():
    def __init__(
        self,
        participants: List[Union[ArtificialIntelligence, Callable]],
        instances_per_round: int = 4,
        name: str = "unnamed",
        skip: int = 0,
        seed_offset: int = 0
    ):
        self.participants = participants
        self.instances_per_round = instances_per_round
        self.name = name
        self.skip = skip
        self.seed_offset = seed_offset

Die Hilfsfunktion `save` speichert die Ergebnisse einer Runde in einer menschenlesbaren Datei, die nach dem Schema `round-NAME-ID.txt` benannt ist. Die Funktion erwartet drei Parameter:
* `idx` ist die Nummer der aktuellen Runde (wird um eins erhöht, um eine natürliche Zählung zu verwenden);
* `rnd` ist die aktuelle Runde;
* `results` ist die Liste der Ergebnisse, die gespeichert werden soll.

In [ ]:
def save(self: Tournament, idx: int, rnd: Round, results: List[MatchResult]):
    path = f"round-{self.name}-{idx+1}.txt"
    with open(path, "w") as file:
        file.write(f"Round: {idx+1}\n")
        
        file.write(f"\nPlayer:\n")
        white = rnd.white() if callable(rnd.white) else rnd.white
        black = rnd.black() if callable(rnd.black) else rnd.black
        file.write(f"  white: {white}\n")
        file.write(f"  black: {black}\n")
        
        file.write(f"\nResult:\n")
        file.write(f"  draw : {sum(result.winner==' ' for result in results)}\n")
        file.write(f"  white: {sum(result.winner=='w' for result in results)}\n")
        file.write(f"  black: {sum(result.winner=='b' for result in results)}\n")
        
        for midx, result in enumerate(results):
            file.write(f"\nMatch {midx+1}:\n")
            file.write(f"  Winner: '{result.winner}'\n")
            file.write(f"  Reason: {result.reason}\n")
            file.write(f"  Log:\n")
            for sidx, state in enumerate(result.log):
                file.write(f"    {sidx+1: >3}. {state}\n")
    return path

Tournament.save = save
del save

Die Funktion `play` startet das Tunier und speichert per Hilfsfunktion `save` die Ergebnisse in einer Textdatei ab. Die Laufzeit kann gegebenenfalls mehrere Stunden lang sein. Mit Hilfe der Bibliotheken `time` und `tqdm` wird der Fortschitt angezeigt.

Hierzu werden alle möglichen Konstellationen der Teilnehmer erzeugt, sodass jeder Teilnehmer gegen jeden anderen Teilnehmer, sowohl als *weiß* als auch als *schwarz*, spielt. Spiele gegen sich selbst werden nicht durchgeführt.

In [ ]:
import time, tqdm
def play(self: Tournament):
    rounds = list(enumerate(
        Round(a, b, self.instances_per_round, self.seed_offset)
        for a in self.participants
        for b in self.participants
        if a != b
    ))
    
    for idx, rnd in tqdm.tqdm(rounds):
        if idx < self.skip:
            print(f"Round {idx+1: >2}/{len(rounds)} was skipped")
            continue
        
        start = time.time()
        results = rnd.play()
        end = time.time()
        print(f"Round {idx+1: >2}/{len(rounds)} took {end-start}")
        path = self.save(idx, rnd, results)
        print(f" > Saved to {path}")

Tournament.play = play
del play